In [43]:
import os
import sys

from ReadMaestro import maestro
from ReadMaestro import parse_pursuit_trials
paths = [p for p in sys.path if os.path.exists(p)]
print(paths)

from importlib import reload
reload(maestro)
reload(parse_pursuit_trials)

['/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '/home/seth/.local/lib/python3.9/site-packages', '/usr/local/lib/python3.9/dist-packages', '/usr/lib/python3/dist-packages']


<module 'ReadMaestro.parse_pursuit_trials' from '/home/seth/analysis/ReadMaestro/ReadMaestro/parse_pursuit_trials.py'>

In [2]:
file_path = '/home/seth/Projects/DynamicCoherencePhysiology/ar/20220404a/ar220404a.0001'
with open(file_path, 'rb') as f:
    data_file = maestro.DataFile.load(f.read(), file_path)

In [5]:
data = maestro.load_directory('/home/seth/Projects/DynamicCoherencePhysiology/test/20191122a')

In [27]:
data[0].header

In [48]:
trialN = len(data)
temp = parse_pursuit_trials.pursuitDataObject()
trialObj = []
for triali in range(trialN):
    trial_str = data[triali].header.trial_name
    temp.name = trial_str[0:5]
    trialObj.append(temp)

print(trialObj[0].name)

Idx = [0, 1]


dPref


In [ ]:
T = [data[i].header.trial_name for i in Idx]
T